<a href="https://colab.research.google.com/github/deanhoperobertson/Named-Enitty-Recognition/blob/master/BiLSTM-CRF%20(6B%20Glove%2050D%20%2B%20Casing%20Features).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo pip install git+https://www.github.com/keras-team/keras-contrib.git
!pip install sklearn_crfsuite
!pip install keras==2.2.4

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-bamb4mz_
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-bamb4mz_
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101064 sha256=70e40282612df9ad1d69f0c0b4adafaf05027c61398f8db8fea23ebbff164329
  Stored in directory: /tmp/pip-ephem-wheel-cache-gnvcx11o/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib
     |████████████████████████████████| 757kB 4.3MB/s 
     |████████████████████████████████| 317kB 2.7MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [2]:
import pandas as pd
import re
import numpy as np
import urllib.request
from sklearn.model_selection import train_test_split

#keras and tensorflow packages
from keras.layers.merge import add
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional,concatenate
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy
from keras_contrib.metrics import crf_accuracy


#evaluation
from sklearn_crfsuite.metrics import flat_classification_report,flat_f1_score,flat_precision_score

Using TensorFlow backend.


In [0]:
#import data from my github repo
train_url = "https://raw.githubusercontent.com/deanhoperobertson/Named-Enitty-Recognition/master/Data/train.txt"
test_url = "https://raw.githubusercontent.com/deanhoperobertson/Named-Enitty-Recognition/master/Data/test.txt"
#valid_url = "https://raw.githubusercontent.com/deanhoperobertson/Named-Enitty-Recognition/master/Data/valid.txt"
train = urllib.request.urlopen(train_url).read()
test = urllib.request.urlopen(test_url).read()
train = train.decode('utf-8')
test = test.decode('utf-8')
#valid = valid.decode('utf-8')

def readstring(filename, meth):
    f = filename.split('\n')
    sentences = []
    sentence = []
    for line in f:
        if len(line) == 0 or line.startswith('-DOCSTART') or line[0] == "\n":
            if len(sentence) > 0:
                sentences.append(sentence)
                sentence = []
            continue
        splits = line.split(' ')
        if meth.lower()=="numbers1":
            sentence.append([hasNumbers1(splits[0]), splits[-1].strip()])
        elif meth.lower()=="numbers2":
            sentence.append([hasNumbers2(splits[0]), splits[-1].strip()])
        else:
            sentence.append([splits[0], splits[-1].strip()])
    if len(sentence) > 0:
        sentences.append(sentence)
        sentence = []
    return sentences

def hasNumbers1(inputString):
    if re.search(r'\d', inputString):
        return "__"
    else:return(inputString)

def hasNumbers2(text):
  if text.isdigit():
      return "1"
  elif re.search(r'\d',text) and re.search(r'\,|\.',text):
      return "1" 
  else:
      if re.search(r'\d', text):
          return(re.sub('\d','D', text))
      else:
          return text

#preproces the txt file
train_data = readstring(train,"Numbers1")
test_data = readstring(test,"Numbers1")

#create corpus
corpus = train_data.copy()
corpus.extend(test_data)

In [0]:
def reformat_data(data,meth):
  if meth.lower() == "data":
    i=0
  else: i=1
  train = []
  output= []
  for sentence in data:
    words=[]
    for x in sentence:
      words.append(x[i])
    train.append(words)

  for i in train:
    string = ' '.join(i)
    output.append(string)
  return output

def get_max_length(corpus):
  length = []
  for sentence in corpus:
    length.append(len(sentence))
  return int(max(length))

def number_of_tags(corpus):
  tags=[]
  for sentence in corpus:
    for tag in sentence:
      tags.append(tag[1])
  return int(len(list(set(tags))))


MAX_LEN = get_max_length(corpus)
N_tags = number_of_tags(corpus)

train = reformat_data(train_data,"data")
test = reformat_data(test_data,"data")


In [0]:
# create a tokenizer
token_word = text.Tokenizer(char_level=False, lower=True, filters="}", oov_token='UNK')
token_word.fit_on_texts(train)

# convert text to sequence of tokens and pad them to ensure equal length vectors 
X_train = sequence.pad_sequences(token_word.texts_to_sequences(train), maxlen=MAX_LEN,padding="post")
X_test = sequence.pad_sequences(token_word.texts_to_sequences(test), maxlen=MAX_LEN,padding="post")

In [0]:
train = reformat_data(train_data,"tags")
test = reformat_data(test_data,"tags")

# create a tokenizer
token_tag = text.Tokenizer(char_level=False, lower=False, filters="}")
token_tag.fit_on_texts(train)

# convert text to sequence of tokens and pad them to ensure equal length vectors 
Y = sequence.pad_sequences(token_tag.texts_to_sequences(train), maxlen=MAX_LEN,padding="post")
# One-Hot encode categorical labels
Y_train = [to_categorical(i, num_classes=N_tags+1) for i in Y]

Y = sequence.pad_sequences(token_tag.texts_to_sequences(test), maxlen=MAX_LEN,padding="post")
# One-Hot encode categorical labels
Y_test = [to_categorical(i, num_classes=N_tags+1) for i in Y]

#add padding 
token_tag.index_word[0]="PAD"
sub_label = list(token_tag.index_word.values())
sub_label.remove('O')
sub_label.remove('PAD')

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [8]:
root_path = "/content/drive/My Drive/glove.6B.50d.txt"
#root_path = "/content/drive/My Drive/glove.6B.100d.txt"
#root_path = "/content/drive/My Drive/glove.6B.200d.txt"
#root_path = "/content/drive/My Drive/glove.6B.300d.txt"
#root_path = "/content/drive/My Drive/glove.42B.300d.txt"
#root_path = "/content/drive/My Drive/wiki-news-300d-1M.vec"

embeddings_index={}
f = open(root_path, encoding = "utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [0]:
#create emedding matrix
EMBEDDING=int(root_path.split(".")[-2][:-1])
word_index = token_word.word_index
embedding_matrix = np.zeros((len(token_word.word_index) + 1, EMBEDDING))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      # words not found in embedding index will be all-zeros.
      embedding_matrix[i] = embedding_vector

In [10]:
checks=[]
words=[]
for i in range(0,len(token_word.word_index)+1):
  if embedding_matrix[i][0] == 0.0:
    checks.append(1)
    words.append(list(token_word.word_index.items())[i-1][0])
  else:
    continue

print("Missing words from Embeddings: %d (%.2f%%)" %(len(checks),(len(checks)/len(token_word.word_index)*100)))

Missing words from Embeddings: 1218 (7.10%)


## Create Casing Embedding

In [0]:
def getCasing(word, caseLookup):   
    casing = 'other'
    
    numDigits = 0
    for char in word:
        if char.isdigit():
            numDigits += 1
            
    digitFraction = numDigits / float(len(word))
    
    if word.isdigit(): #Is a digit
        casing = 'numeric'
    elif digitFraction > 0.5:
        casing = 'mainly_numeric'
    elif word.islower(): #All lower case
        casing = 'allLower'
    elif word.isupper(): #All upper case
        casing = 'allUpper'
    elif word[0].isupper(): #is a title, initial char upper, then all lower
        casing = 'initialUpper'
    elif numDigits > 0:
        casing = 'contains_digit'  
    return caseLookup[casing]

case2Idx = {'numeric': 7, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 
            'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':0}

caseEmbeddings = np.identity(len(case2Idx), dtype='float32')


X_cas = []
for sentence in train_data:
    sent_seq = []
    for i in range(MAX_LEN):
        word_seq = []
        try:
          word_seq.append(getCasing(sentence[i][0],case2Idx))
        except:
          word_seq.append(case2Idx.get('PADDING_TOKEN'))
        sent_seq.append(word_seq)
    X_cas.append(np.array(sent_seq))

## Create Model

In [13]:
# Model definition

words_in = Input(shape=(MAX_LEN,), name="Input_Words")
casing_in = Input(shape=(MAX_LEN,), name="Input_Casings")

word_emb = Embedding(input_dim=len(token_word.word_index) + 1,output_dim=EMBEDDING,
                     weights=[embedding_matrix],input_length=MAX_LEN,
                     trainable=False,mask_zero=True)(words_in)

# casing_emb = TimeDistributed(Embedding(input_dim=caseEmbeddings.shape[0],output_dim=caseEmbeddings.shape[1],
#                        weights=[caseEmbeddings], trainable=False, mask_zero=True))(casing_in)

casing_emb = Embedding(input_dim=caseEmbeddings.shape[0],output_dim=caseEmbeddings.shape[1],
                       weights=[caseEmbeddings], trainable=False, mask_zero=True)(casing_in)

# character LSTM to get word encodings by characters
# cas_enc = TimeDistributed(LSTM(units=20, return_sequences=False, #20
#                                 recurrent_dropout=0.5))(casing_emb) #0.3/0.5

model_2 = concatenate([word_emb,casing_emb])
model_2 = Bidirectional(LSTM(units=300, return_sequences=True,
                           recurrent_dropout=0.1, dropout=0.3))(model_2) 

model_2 = TimeDistributed(Dense(50, activation="relu"))(model_2)
crf = CRF(N_tags+1)  # CRF layer
out = crf(model_2)  # output
model_2 = Model([words_in, casing_in], out)










Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [14]:
from keras import optimizers
adam=optimizers.Adam(clipvalue=0.5)

model_2.compile(optimizer=adam, loss=crf_loss,metrics=[crf_viterbi_accuracy])
model_2.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input_Words (InputLayer)        (None, 124)          0                                            
__________________________________________________________________________________________________
Input_Casings (InputLayer)      (None, 124)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 124, 50)      858250      Input_Words[0][0]                
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 124, 8)       64          Input_Casings[0][0]              
__________________________________________________________________________________________________
concatena

In [0]:
from keras.callbacks import EarlyStopping
from keras.callbacks import Callback

class F1Metrics(Callback):

    def __init__(self, id2label,sub_label,pad_value=0,validation_data=None):
        super(F1Metrics, self).__init__()
        self.id2label = id2label
        self.sublabel = sub_label
        self.validation_data = validation_data
        self.is_fit = validation_data is None

    def convert_idx_to_name(self, y):
        y = [[self.id2label[i] for i in row] for row in y]
        return y

    def predict(self, X, y):
        y_pred = self.model.predict_on_batch(X)
        y_true = np.argmax(y, -1)
        y_pred = np.argmax(y_pred, -1)
        y_true = self.convert_idx_to_name(y_true)
        y_pred = self.convert_idx_to_name(y_pred)
        return y_true, y_pred

    def score(self, y_true, y_pred):
        score = flat_f1_score(y_true, y_pred,average='micro',labels=sub_label)
        print(len(y_pred))
        print(' - f1: %.2f' %(score))
        return score

    def on_epoch_end(self, epoch, logs={}):
        if self.is_fit:
            self.on_epoch_end_fit(epoch, logs)
        else:
            self.on_epoch_end_fit_generator(epoch, logs)

    def on_epoch_end_fit(self, epoch, logs={}):
        X = self.validation_data[0]
        y = self.validation_data[1]
        y_true, y_pred = self.predict(X, y)
        score_val = self.score(y_true, y_pred)
        logs['f1_val'] = score_val


    def on_epoch_end_fit_generator(self, epoch, logs={}):
        y_true = []
        y_pred = []
        for X, y in self.validation_data:
            y_true_batch, y_pred_batch = self.predict(X, y)
            y_true.extend(y_true_batch)
            y_pred.extend(y_pred_batch)
        score = self.score(y_true, y_pred)
        logs['f1'] = score

f1score = F1Metrics(token_tag.index_word, sub_label)

In [16]:
%%time
from keras.callbacks import EarlyStopping
from keras.callbacks import Callback
BATCH_SIZE = 200
EPOCHS=20

early_stopping = EarlyStopping(monitor='val_loss', patience=2)

history = model_2.fit([X_train, np.array(X_cas).reshape((len(X_cas), MAX_LEN))],
                      np.array(Y_train),
                      batch_size=BATCH_SIZE, 
                      epochs=EPOCHS, 
                      validation_split=0.2, 
                       #validation_data = (X_valid,np.array(Y_valid)),
                      verbose=1,callbacks=[early_stopping])



Train on 11232 samples, validate on 2809 samples
Epoch 1/20
11232/11232 [==============================] - 51s 5ms/step - loss: 40.6858 - crf_viterbi_accuracy: 0.8186 - val_loss: 35.3623 - val_crf_viterbi_accuracy: 0.9147
Epoch 2/20
11232/11232 [==============================] - 48s 4ms/step - loss: 40.1891 - crf_viterbi_accuracy: 0.9337 - val_loss: 35.2418 - val_crf_viterbi_accuracy: 0.9483
Epoch 3/20
11232/11232 [==============================] - 47s 4ms/step - loss: 40.1210 - crf_viterbi_accuracy: 0.9522 - val_loss: 35.2022 - val_crf_viterbi_accuracy: 0.9620
Epoch 4/20
11232/11232 [==============================] - 46s 4ms/step - loss: 40.0943 - crf_viterbi_accuracy: 0.9594 - val_loss: 35.1930 - val_crf_viterbi_accuracy: 0.9634
Epoch 5/20
11232/11232 [==============================] - 46s 4ms/step - loss: 40.0748 - crf_viterbi_accuracy: 0.9636 - val_loss: 35.1747 - val_crf_viterbi_accuracy: 0.9670
Epoch 6/20
11232/11232 [==============================] - 47s 4ms/step - loss: 40.06

## Predict On Training Set

In [17]:
%%time
pred = model_2.predict([X_train, np.array(X_cas).reshape((len(X_cas), MAX_LEN))], verbose=1)

14041/14041 [==============================] - 125s 9ms/step
CPU times: user 3min 8s, sys: 26.5 s, total: 3min 34s
Wall time: 2min 4s


In [18]:
# TRain Eval
preds = np.argmax(pred, axis=-1)
y_tr_true = np.argmax(Y_train, -1)

# Convert the index to tag
pred_tag = [[token_tag.index_word[i] for i in row] for row in preds]
y_tr_true_tag = [[token_tag.index_word[i] for i in row] for row in y_tr_true]

report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag,labels=sub_label)
print(report)
#F1 Score
score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='micro',labels=sub_label)
print(score)  

              precision    recall  f1-score   support

       I-PER       0.97      0.98      0.98     11128
       I-ORG       0.95      0.91      0.93     10001
       I-LOC       0.94      0.96      0.95      8286
      I-MISC       0.88      0.89      0.89      4556
      B-MISC       0.00      0.00      0.00        37
       B-ORG       1.00      1.00      1.00        24
       B-LOC       1.00      0.64      0.78        11

   micro avg       0.95      0.94      0.94     34043
   macro avg       0.82      0.77      0.79     34043
weighted avg       0.95      0.94      0.94     34043

0.9438248596726432


## Predict On Test Set

In [19]:
%%time
X_cas_t = []
for sentence in test_data:
    sent_seq = []
    for i in range(MAX_LEN):
        word_seq = []
        try:
          word_seq.append(getCasing(sentence[i][0],case2Idx))
        except:
          word_seq.append(case2Idx.get('PADDING_TOKEN'))
        sent_seq.append(word_seq)
    X_cas_t.append(np.array(sent_seq))

pred = model_2.predict([X_test, np.array(X_cas_t).reshape((len(X_cas_t), MAX_LEN))], verbose=1)

3453/3453 [==============================] - 30s 9ms/step
CPU times: user 45.7 s, sys: 6.56 s, total: 52.3 s
Wall time: 30.4 s


In [20]:
# Test Eval
#pred_cat = model.predict(X_tr)
preds = np.argmax(pred, axis=-1)
y_tr_true = np.argmax(Y_test, -1)

# Convert the index to tag
pred_tag = [[token_tag.index_word[i] for i in row] for row in preds]
y_tr_true_tag = [[token_tag.index_word[i] for i in row] for row in y_tr_true]
                 
                 
report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag, labels=sub_label)
print(report)

score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='micro', labels=sub_label)
print(score)    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       I-PER       0.95      0.92      0.93      2773
       I-ORG       0.85      0.79      0.82      2491
       I-LOC       0.86      0.88      0.87      1919
      I-MISC       0.61      0.79      0.69       909
      B-MISC       0.00      0.00      0.00         9
       B-ORG       0.00      0.00      0.00         5
       B-LOC       0.00      0.00      0.00         6

   micro avg       0.85      0.85      0.85      8112
   macro avg       0.47      0.48      0.47      8112
weighted avg       0.86      0.85      0.85      8112

0.8520862778836109


## Underfitting

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
# summarize history for accuracy
plt.subplot(1,1,1)
plt.plot(history.history['crf_viterbi_accuracy'])
plt.plot(history.history['val_crf_viterbi_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'])
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='center right')
plt.show()